In [1]:
from vivarium import InteractiveContext

In [2]:
import pandas as pd
import numpy as np

* go on the zenon repo
* run 'make_specs'
* Then you go into ipython/jupyter notebook
* import InteractiveContext
* p = path to specifications yaml file
* make simulation by running:
* sim = InteractiveContext(p)
* this takes a couple minutes to build

To get sim outputs:
* you need to call sim.finalize

After running, we care about
sim.get_population() -> returns DataFrame of state table (state of every simulant in the sim)
sim.list_values() -> returns list of all values stored in pipelines
sim.get_value() (of the pipeline name (index))
ex: sim.get_value(pipeline_name)(pop.index)

ex: pop = sim.get_population()

In [7]:
sim.list_values()

['metrics',
 'cause_specific_mortality_rate',
 'mortality_rate',
 'acute_myocardial_infarction.incidence_rate.paf',
 'post_myocardial_infarction_to_acute_myocardial_infarction.transition_rate.paf',
 'acute_ischemic_stroke.incidence_rate.paf',
 'post_ischemic_stroke_to_acute_ischemic_stroke.transition_rate.paf',
 'acute_myocardial_infarction.incidence_rate',
 'acute_myocardial_infarction.dwell_time',
 'acute_myocardial_infarction.disability_weight',
 'disability_weight',
 'acute_myocardial_infarction.excess_mortality_rate',
 'acute_myocardial_infarction.excess_mortality_rate.population_attributable_fraction',
 'post_myocardial_infarction.dwell_time',
 'post_myocardial_infarction.disability_weight',
 'post_myocardial_infarction.excess_mortality_rate',
 'post_myocardial_infarction.excess_mortality_rate.population_attributable_fraction',
 'post_myocardial_infarction_to_acute_myocardial_infarction.transition_rate',
 'acute_ischemic_stroke.incidence_rate',
 'acute_ischemic_stroke.dwell_time'

In [35]:
p = f'/ihme/homes/santonis/notebooks/sanofi_zenon/brazil.yaml'
sim = InteractiveContext(p)
pop = sim.get_population()


2020-04-23 10:31:15.555 | DEBUG    | vivarium.framework.values:register_value_modifier:373 - Registering metrics.1.population_manager.metrics as modifier to metrics
2020-04-23 10:31:20.425 | DEBUG    | vivarium.framework.artifact.manager:_load_artifact:66 - Running simulation from artifact located at /share/costeffectiveness/artifacts/vivarium_csu_zenon/brazil.hdf.
2020-04-23 10:31:20.427 | DEBUG    | vivarium.framework.artifact.manager:_load_artifact:67 - Artifact base filter terms are ['draw == 0', "location == 'Brazil' | location == 'Global'"].
2020-04-23 10:31:20.429 | DEBUG    | vivarium.framework.artifact.manager:_load_artifact:68 - Artifact additional filter terms are None.
2020-04-23 10:31:21.265 | DEBUG    | vivarium.framework.values:_register_value_producer:323 - Registering value pipeline cause_specific_mortality_rate
2020-04-23 10:31:21.267 | DEBUG    | vivarium.framework.values:_register_value_producer:323 - Registering value pipeline mortality_rate
2020-04-23 10:31:22.090

In [42]:
pop.columns

Index(['tracked', 'had_adverse_event', 'previous_chronic_kidney_disease',
       'previous_diabetes_mellitus', 'previous_ischemic_stroke',
       'previous_ischemic_heart_disease', 'years_lived_with_disability',
       'years_of_life_lost', 'cause_of_death', 'entrance_time', 'location',
       'age', 'alive', 'sex', 'exit_time',
       'ldlc_treatment_adherence_propensity', 'ischemic_stroke',
       'post_ischemic_stroke_event_count', 'post_ischemic_stroke_event_time',
       'acute_ischemic_stroke_event_time', 'acute_ischemic_stroke_event_count',
       'susceptible_to_ischemic_stroke_event_time',
       'susceptible_to_ischemic_stroke_event_count', 'ischemic_heart_disease',
       'post_myocardial_infarction_event_time',
       'post_myocardial_infarction_event_count',
       'acute_myocardial_infarction_event_count',
       'acute_myocardial_infarction_event_time',
       'susceptible_to_ischemic_heart_disease_event_count',
       'susceptible_to_ischemic_heart_disease_event_time',


In [3]:
locations = ['brazil', 'china', 'italy', 'france', 'russian_federation', 'spain']

In [4]:
def get_init_data(locations):
    all_countries = pd.DataFrame([])
    for location in locations:
        p = f'/ihme/homes/santonis/notebooks/sanofi_zenon/{location}.yaml'
        sim = InteractiveContext(p)
        pop = sim.get_population()
        df = pop[['ischemic_heart_disease', 'ischemic_stroke', 'ldlc_treatment_category', 'age']] 
        df['ldl_c'] = sim.get_value('high_ldl_cholesterol.exposure')(pop.index)
        df['fpg'] = sim.get_value('high_fasting_plasma_glucose_continuous.exposure')(pop.index)
        df['sbp'] = sim.get_value('high_systolic_blood_pressure.exposure')(pop.index)
        df['ikf'] = sim.get_value('impaired_kidney_function.exposure')(pop.index)
        df['adherent'] = sim.get_value('ldlc_treatment_adherence')(pop.index)
        df['location'] = f'{location}'
        all_countries = all_countries.append(df)
    return all_countries

In [5]:
df = get_init_data(locations)

2020-04-24 14:43:41.525 | DEBUG    | vivarium.framework.values:register_value_modifier:373 - Registering metrics.1.population_manager.metrics as modifier to metrics
2020-04-24 14:43:41.690 | DEBUG    | vivarium.framework.artifact.manager:_load_artifact:66 - Running simulation from artifact located at /share/costeffectiveness/artifacts/vivarium_csu_zenon/brazil.hdf.
2020-04-24 14:43:41.692 | DEBUG    | vivarium.framework.artifact.manager:_load_artifact:67 - Artifact base filter terms are ['draw == 0', "location == 'Brazil' | location == 'Global'"].
2020-04-24 14:43:41.693 | DEBUG    | vivarium.framework.artifact.manager:_load_artifact:68 - Artifact additional filter terms are None.
2020-04-24 14:43:44.119 | DEBUG    | vivarium.framework.values:_register_value_producer:323 - Registering value pipeline cause_specific_mortality_rate
2020-04-24 14:43:44.121 | DEBUG    | vivarium.framework.values:_register_value_producer:323 - Registering value pipeline mortality_rate
2020-04-24 14:43:46.317

## Table 2 validation

In [11]:
table_2_df = pd.read_csv('/ihme/homes/santonis/notebooks/sanofi_zenon/prob_rx_given_high_ldlc.csv')
table_2_df['location'] = table_2_df.location.str.replace('russia', 'russian_federation')
table_2_df = table_2_df.rename(columns={'mean_value':'table_2_mean_value', 'sd_value':'table_2_sd_value'})

In [12]:
def get_table_2_validations(df, locations, table_2_df):    
    init_df = pd.DataFrame([])
    shared_cols = ['location']
    for location in locations:
        df_loop = df[(df.location == location)]

        df_loop['output_mean_value'] = len(df_loop[(df_loop.ldlc_treatment_category != 'none')]) / (len(df_loop[(df_loop.ldlc_treatment_category == 'none') & (df_loop.ldl_c>=5.0)]) + len(df_loop[(df_loop.ldlc_treatment_category != 'none')]))
        
        init_df = init_df.append(df_loop)
    init_df = init_df[['location','output_mean_value']]
    init_df = init_df.drop_duplicates()
    init_df['output_mean_value'] = init_df.output_mean_value.astype(float)
    init_df = pd.merge(init_df, table_2_df, left_on=shared_cols, right_on=shared_cols)
    init_df= init_df.round(2)
    return init_df



In [13]:
#with 100_000 sim
get_table_2_validations(df, locations, table_2_df)

/share/code/santonis/miniconda/envs/vivarium_csu_zenon/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,location,output_mean_value,table_2_mean_value,table_2_sd_value
0,brazil,0.53,0.42,0.19
1,china,0.24,0.32,0.18
2,italy,0.72,0.53,0.21
3,france,0.45,0.50,0.12
4,russian_federation,0.79,0.39,0.28
5,spain,0.47,0.34,0.10


In [46]:
get_table_2_validations(df, locations, table_2_df)

/share/code/santonis/miniconda/envs/vivarium_csu_zenon/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,location,output_mean_value,table_2_mean_value,table_2_sd_value
0,brazil,0.54,0.42,0.19
1,china,0.23,0.32,0.18
2,italy,0.73,0.53,0.21
3,france,0.45,0.50,0.12
4,spain,0.47,0.34,0.10


## Table 6 validation

In [14]:
table_6_df = pd.read_csv('/ihme/homes/santonis/notebooks/sanofi_zenon/current_rx.csv')
table_6_df['location'] = table_6_df.location.str.replace('russia', 'russian_federation')

table_6_df = table_6_df.rename(columns={'mean_value':'table_6_mean_value', 'sd_value':'table_6_sd_value'})

In [15]:
def get_table_6_validations(df, locations, table_6_df):    
    init_df = pd.DataFrame([])
    shared_cols = ['location', 'current_prescription']
    for location in locations:
        df_loop = df[(df.location == location)]

        df_loop['% on ezetimibe'] = len(df_loop[(df_loop.ldlc_treatment_category == 'ezetimibe')])/ len(df_loop[(df_loop.ldlc_treatment_category != 'none')])
        df_loop['% on fibrates'] = len(df_loop[df_loop.ldlc_treatment_category == 'fibrates']) / len(df_loop[(df_loop.ldlc_treatment_category != 'none')])
        df_loop['% on high potency statin'] = len(df_loop[(df_loop.ldlc_treatment_category == 'high_potency_statin_low_dose')]) / len(df_loop[(df_loop.ldlc_treatment_category != 'none')])
        df_loop['% on low potency statin'] = len(df_loop[(df_loop.ldlc_treatment_category.str.contains('low_potency_statin_'))]) / len(df_loop[(df_loop.ldlc_treatment_category != 'none')])
        init_df = init_df.append(df_loop)
   
    init_df = pd.melt(init_df, id_vars=['location'])
    init_df = init_df[(init_df.variable.str.contains('%'))]
    init_df = init_df.drop_duplicates()
    init_df = init_df.rename(columns={'variable':'current_prescription', 'value':'output_mean_value'})
    init_df['output_mean_value'] = init_df.output_mean_value.astype(float)
    init_df = pd.merge(init_df, table_6_df, left_on=shared_cols, right_on=shared_cols)
    init_df= init_df.round(2)
    return init_df

In [12]:
pct_high_statin = len(df[(df.ldlc_treatment_category == 'high_potency_statin_low_dose')]) / len(df[(df.ldlc_treatment_category != 'none')])


In [13]:
pct_high_statin

0.5127263827704356

In [16]:
#with 100_000 sim
get_table_6_validations(df, locations, table_6_df)


/share/code/santonis/miniconda/envs/vivarium_csu_zenon/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/share/code/santonis/miniconda/envs/vivarium_csu_zenon/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/share/code/santonis/miniconda/envs/vivarium_csu_zenon/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

,location,current_prescription,output_mean_value,table_6_mean_value,table_6_sd_value
0,brazil,% on ezetimibe,0.05,0.10,0.10
1,china,% on ezetimibe,0.02,0.00,0.01
2,italy,% on ezetimibe,0.03,0.10,0.10
3,france,% on ezetimibe,0.06,0.04,0.04
4,russian_federation,% on ezetimibe,0.00,0.00,0.00
5,spain,% on ezetimibe,0.11,0.10,0.09
6,brazil,% on fibrates,0.06,0.14,0.12
7,china,% on fibrates,0.09,0.09,0.05
8,italy,% on fibrates,0.01,0.01,0.00
9,france,% on fibrates,0.11,0.13,0.07


In [21]:
get_table_6_validations(df, locations, table_6_df)


/share/code/santonis/miniconda/envs/vivarium_csu_zenon/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/share/code/santonis/miniconda/envs/vivarium_csu_zenon/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/share/code/santonis/miniconda/envs/vivarium_csu_zenon/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

,location,current_prescription,output_mean_value,table_6_mean_value,table_6_sd_value
0,brazil,% on ezetimibe,0.05,0.10,0.10
1,china,% on ezetimibe,0.01,0.00,0.01
2,italy,% on ezetimibe,0.03,0.10,0.10
3,france,% on ezetimibe,0.07,0.04,0.04
4,spain,% on ezetimibe,0.10,0.10,0.09
5,brazil,% on fibrates,0.07,0.14,0.12
6,china,% on fibrates,0.09,0.09,0.05
7,italy,% on fibrates,0.00,0.01,0.00
8,france,% on fibrates,0.11,0.13,0.07
9,spain,% on fibrates,0.00,0.00,0.00


## Table 8 validation

In [17]:
table_8_doc = pd.read_csv('/ihme/homes/santonis/notebooks/sanofi_zenon/dist_therapy_type.csv')
table_8_doc['location'] = table_8_doc.location.str.replace('russia', 'russian_federation')
table_8_doc = table_8_doc.rename(columns={'mean_value':'table_8_mean', 'sd_value':'table_8_sd'})

In [18]:
def get_table_8_validations(df, locations, table_8_doc):    
    init_df = pd.DataFrame([])
    shared_cols = ['location', 'therapy_type']
    for location in locations:
        df_loop = df[(df.location == location)]

        df_loop['% on monotherapy'] = len(df_loop[(df_loop.ldlc_treatment_category.str.contains(r'^(?:(?!_multi).)*$')) & (df_loop.ldlc_treatment_category.str.contains(r'^(?:(?!_fdc).)*$')) & (df_loop.ldlc_treatment_category != 'none')]) / len(df_loop[(df_loop.ldlc_treatment_category != 'none')])
        df_loop['% on multi-drugs'] = len(df_loop[((df_loop.ldlc_treatment_category.str.contains('multi')) | (df_loop.ldlc_treatment_category.str.contains('fdc')))]) / len(df_loop[(df_loop.ldlc_treatment_category != 'none')])
#         df_loop['% on FDC if multi drug'] = len(df_loop[(df_loop.ldlc_treatment_category.str.contains('fdc'))]) / len(df_loop[(df_loop.ldlc_treatment_category.str.contains('multi')) | (df_loop.ldlc_treatment_category.str.contains('fdc'))])
        df_loop['% on FDC if multi drug'] = len(df_loop[(df_loop.ldlc_treatment_category.str.contains('fdc'))]) / len(df_loop[(df_loop.ldlc_treatment_category.str.contains('multi')) | (df_loop.ldlc_treatment_category.str.contains('fdc'))])

        init_df = init_df.append(df_loop)
    init_df = pd.melt(init_df, id_vars=['location'])
    init_df = init_df[(init_df.variable.str.contains('%'))]
    init_df = init_df.drop_duplicates()
    init_df = init_df.rename(columns={'variable':'therapy_type', 'value':'output_mean_value'})
    init_df['output_mean_value'] = init_df.output_mean_value.astype(float)
    init_df = pd.merge(init_df, table_8_doc, left_on=shared_cols, right_on=shared_cols)
    init_df= init_df.round(2)
    return init_df


In [19]:
#with 100_000 sim
get_table_8_validations(df, locations, table_8_doc)

/share/code/santonis/miniconda/envs/vivarium_csu_zenon/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/share/code/santonis/miniconda/envs/vivarium_csu_zenon/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/share/code/santonis/miniconda/envs/vivarium_csu_zenon/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

,location,therapy_type,output_mean_value,table_8_mean,table_8_sd
0,brazil,% on monotherapy,0.77,0.70,0.00
1,china,% on monotherapy,0.99,0.98,0.00
2,italy,% on monotherapy,0.81,0.78,0.02
3,france,% on monotherapy,0.88,0.85,0.00
4,russian_federation,% on monotherapy,0.97,0.96,0.08
5,spain,% on monotherapy,0.76,0.68,0.00
6,brazil,% on multi-drugs,0.23,0.30,0.20
7,china,% on multi-drugs,0.01,0.02,0.01
8,italy,% on multi-drugs,0.19,0.22,0.14
9,france,% on multi-drugs,0.12,0.15,0.10


In [24]:
get_table_8_validations(df, locations, table_8_doc)

/share/code/santonis/miniconda/envs/vivarium_csu_zenon/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/share/code/santonis/miniconda/envs/vivarium_csu_zenon/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/share/code/santonis/miniconda/envs/vivarium_csu_zenon/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

,location,therapy_type,output_mean_value,table_8_mean,table_8_sd
0,brazil,% on monotherapy,0.76,0.70,0.00
1,china,% on monotherapy,1.00,0.98,0.00
2,italy,% on monotherapy,0.81,0.78,0.02
3,france,% on monotherapy,0.88,0.85,0.00
4,russian_federation,% on monotherapy,0.97,0.96,0.08
5,spain,% on monotherapy,0.76,0.68,0.00
6,brazil,% on multi-drugs,0.24,0.30,0.20
7,china,% on multi-drugs,0.00,0.02,0.01
8,italy,% on multi-drugs,0.19,0.22,0.14
9,france,% on multi-drugs,0.12,0.15,0.10


## Table 4 -- Adherence

In [20]:
table_4_df = pd.read_csv('/ihme/homes/santonis/notebooks/sanofi_zenon/adherence_parameters.csv')
table_4_df = table_4_df.rename(columns={'mean_value':'table_4_mean', 'sd_value':'table_4_sd'})
table_4_df['location'] = table_4_df.location.str.replace('russia', 'russian_federation')


In [23]:
def get_table_4_validations(df, locations, table_4_df):    
    init_df = pd.DataFrame([])
    shared_cols = ['location', 'adherence_parameter']
    for location in locations:
        df_loop = df[(df.location == location)]
        
        df_loop['Adherence - one pill, no MI'] = len(df_loop[(df_loop.ldlc_treatment_category.str.contains(r'^(?:(?!_multi).)*$')) & 
                            (df_loop.ldlc_treatment_category != 'none') & (df_loop.ischemic_heart_disease == 'susceptible_to_ischemic_heart_disease') 
                            & (df_loop.ischemic_stroke == 'susceptible_to_ischemic_stroke') & (df_loop.adherent == True)]) / len(df_loop[(df_loop.ldlc_treatment_category.str.contains(r'^(?:(?!_multi).)*$') & 
                            (df_loop.ldlc_treatment_category != 'none') & (df_loop.ischemic_heart_disease == 'susceptible_to_ischemic_heart_disease') 
                            & (df_loop.ischemic_stroke == 'susceptible_to_ischemic_stroke'))])

        multi_pill_no_mi = len(df_loop[(df_loop.ldlc_treatment_category.str.contains('multi') 
                                & (df_loop.ldlc_treatment_category != 'none') 
                                & (df_loop.ischemic_heart_disease == 'susceptible_to_ischemic_heart_disease') 
                                & (df_loop.ischemic_stroke == 'susceptible_to_ischemic_stroke'))])
        if multi_pill_no_mi > 0:
            df_loop['Adherence - multi-pill, no MI'] = len(df_loop[(df_loop.ldlc_treatment_category.str.contains('multi') 
                                & (df_loop.ischemic_heart_disease == 'susceptible_to_ischemic_heart_disease') 
                                & (df_loop.ischemic_stroke == 'susceptible_to_ischemic_stroke') 
                                & (df_loop.adherent == True))]) / multi_pill_no_mi
        else:
            df_loop['Adherence - multi-pill, no MI'] = 0
            
        one_pill_mi = len(df_loop[(df_loop.ldlc_treatment_category.str.contains(r'^(?:(?!_multi).)*$')) & 
                            (df_loop.ldlc_treatment_category != 'none') & (df_loop.ischemic_heart_disease != 'susceptible_to_ischemic_heart_disease') & (df_loop.ischemic_stroke != 'susceptible_to_ischemic_stroke')])
        if one_pill_mi > 0:
            df_loop['Adherence after MI (one pill)'] = len(df_loop[(df_loop.ldlc_treatment_category.str.contains(r'^(?:(?!_multi).)*$')) & 
                            (df_loop.ldlc_treatment_category != 'none') & ((df_loop.ischemic_heart_disease != 'susceptible_to_ischemic_heart_disease') 
                            | (df_loop.ischemic_stroke != 'susceptible_to_ischemic_stroke'))
                            & (df_loop.adherent == True)]) / len(df_loop[(df_loop.ldlc_treatment_category.str.contains(r'^(?:(?!_multi).)*$')) & 
                            (df_loop.ldlc_treatment_category != 'none') & ((df_loop.ischemic_heart_disease != 'susceptible_to_ischemic_heart_disease') | (df_loop.ischemic_stroke != 'susceptible_to_ischemic_stroke'))])
        
        
        else:
            df_loop['Adherence after MI (one pill)'] = 0
            
        adherent_mi_mult_pill = df_loop[((df_loop.ldlc_treatment_category.str.contains('multi')) 
                                        & (df_loop.ischemic_heart_disease != 'susceptible_to_ischemic_heart_disease')) 
                                        | ((df_loop.ldlc_treatment_category.str.contains('multi')) 
                                        & (df_loop.ischemic_stroke != 'susceptible_to_ischemic_stroke'))]
        
        pop_multi_pill_mi = len(df_loop[(df_loop.ldlc_treatment_category.str.contains('multi')) 
                                        & ((df_loop.ischemic_heart_disease != 'susceptible_to_ischemic_heart_disease') 
                                        | (df_loop.ischemic_stroke != 'susceptible_to_ischemic_stroke'))])
        if pop_multi_pill_mi > 0.0:
#             df_loop['Adherence after MI (multi-pill)'] = len(df_loop[((df_loop.ldlc_treatment_category.str.contains('multi')) & (df_loop.ischemic_heart_disease != 'susceptible_to_ischemic_heart_disease')& (df_loop.adherent == True)) | ((df_loop.ischemic_stroke != 'susceptible_to_ischemic_stroke') & (df_loop.ldlc_treatment_category.str.contains('multi')) & (df_loop.adherent == True))]) 
           
            df_loop['Adherence after MI (multi-pill)'] = len(adherent_mi_mult_pill) / pop_multi_pill_mi

        else:
            df_loop['Adherence after MI (multi-pill)'] = 0.0
            
        init_df = init_df.append(df_loop)
    
    init_df = pd.melt(init_df, id_vars=['location'])
    init_df = init_df[(init_df.variable.str.contains('Adherence'))]
    init_df = init_df.drop_duplicates()
    init_df = init_df.rename(columns={'variable':'adherence_parameter', 'value':'output_mean_value'})
    init_df['output_mean_value'] = init_df.output_mean_value.astype(float)
    init_df = pd.merge(init_df, table_4_df, left_on=shared_cols, right_on=shared_cols)
    init_df= init_df.round(2)
    return init_df


In [24]:
get_table_4_validations(df, locations, table_4_df)

/share/code/santonis/miniconda/envs/vivarium_csu_zenon/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/share/code/santonis/miniconda/envs/vivarium_csu_zenon/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/share/code/santonis/miniconda/envs/vivarium_csu_zenon/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

,location,adherence_parameter,output_mean_value,table_4_mean,table_4_sd
0,brazil,"Adherence - one pill, no MI",0.38,0.55,0.11
1,china,"Adherence - one pill, no MI",0.53,0.49,0.10
2,italy,"Adherence - one pill, no MI",0.42,0.43,0.09
3,france,"Adherence - one pill, no MI",0.73,0.66,0.14
4,russian_federation,"Adherence - one pill, no MI",0.31,0.38,0.08
5,spain,"Adherence - one pill, no MI",0.65,0.70,0.15
6,brazil,"Adherence - multi-pill, no MI",0.18,0.29,0.06
7,china,"Adherence - multi-pill, no MI",0.17,0.23,0.05
8,italy,"Adherence - multi-pill, no MI",0.16,0.17,0.04
9,france,"Adherence - multi-pill, no MI",0.41,0.40,0.09


In [40]:
test_df = df[(df.ldlc_treatment_category.str.contains('multi') & (df.ldlc_treatment_category != 'none') 
                                & (df.ischemic_heart_disease == 'susceptible_to_ischemic_heart_disease') & (df.ischemic_stroke == 'susceptible_to_ischemic_stroke'))]


In [41]:
test_df['location'].unique()

array(['brazil', 'italy', 'france', 'russian_federation', 'spain'],
      dtype=object)